In [1]:
#pip install rdkit-pypi 
""" установка rdkit """

' установка rdkit '

In [2]:
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
import numpy as np
from tqdm.notebook import tqdm


import warnings
warnings.filterwarnings("ignore")
#Автор классов реализации расчёта дискриптров "https://drzinph.com/"

In [3]:
class RDKit_2D:
    def __init__(self, smiles):
        self.mols = []
        for i in tqdm(smiles):
            try:
                self.mols.append(Chem.MolFromSmiles(i))
            except:
                self.mols.append(None)
                
        self.smiles = smiles

    def compute_2Drdkit(self, name):
        rdkit_2d_desc = []
        calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
        header = calc.GetDescriptorNames()
        for i in tqdm(range(len(self.mols))):
            if self.mols[i] != None:
                ds = calc.CalcDescriptors(self.mols[i])
            else:
                ds = [None for i in range(len(Descriptors._descList))]
            rdkit_2d_desc.append(ds)
        df = pd.DataFrame(rdkit_2d_desc,columns=header)
        df.insert(loc=0, column= name, value=self.smiles)
        df.to_csv(name +'_RDKit_2D.tsv', index=False, sep = "\t")
        #return df

In [4]:
class ECFP6:
    def __init__(self, smiles):
        self.mols = []
        for i in tqdm(smiles):
            try:
                self.mols.append(Chem.MolFromSmiles(i))
            except:
                self.mols.append(None)
        self.smiles = smiles

    def mol2fp(self, mol, radius = 3):
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius = radius)
        array = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, array)
        return array

    def compute_ECFP6(self, name):
        bit_headers = ['bit' + str(i) for i in range(2048)]
        arr = np.empty((0,2048), int).astype(int)
        for i in tqdm(self.mols):
            if i != None:
                fp = self.mol2fp(i)
            else:
                fp = [0 for i in range(2048)]
            arr = np.vstack((arr, fp))
        df_ecfp6 = pd.DataFrame(np.asarray(arr).astype(int),columns=bit_headers)
        df_ecfp6.insert(loc=0, column= name, value=self.smiles)
        df_ecfp6.to_csv(name +'_ECFP6.tsv', index=False, sep = "\t")
        #return df_ecfp6

In [5]:
class MACCS:
    def __init__(self, smiles):

        self.mols = []
        for i in tqdm(smiles):
            try:
                self.mols.append(Chem.MolFromSmiles(i))
            except:
                self.mols.append(None)
        self.smiles = smiles

    def compute_MACCS(self, name):
        MACCS_list = []
        header = ['bit' + str(i) for i in range(167)]
        for i in tqdm(range(len(self.mols))):
            if self.mols[i] != None:
                ds = list(MACCSkeys.GenMACCSKeys(self.mols[i]).ToBitString())
            else:
                ds = [0 for i in range(167)]
                
            MACCS_list.append(ds)
        df = pd.DataFrame(MACCS_list,columns=header)
        df.insert(loc=0, column= name, value=self.smiles)
        df.to_csv(name +'_MACCS.tsv', index=False, sep = "\t")
        #return df

In [5]:
data = pd.read_csv("reaction_preparation.tsv", sep = "\t")
data

,reactions,interm_0,interm_1,interm_2,interm_3,interm_4,interm_5,interm_6,interm_7,interm_8,interm_9,agents_0,agents_1,agents_2,product_0,product_1,Yield
0,[CH2:1]([S:3][C:4]1[CH:26]=[CH:25][C:7]2[N:8](...,O[C:12]1([CH2:13][C:14](=[O:15])[O:16][CH2:17]...,Cl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCO,NaN,NaN,[CH2:1]([CH3:2])[S:3][C:4]1=[CH:5][C:6]2=[C:7]...,NaN,82.0
1,[CH2:1]([S:3][C:4]1[CH:25]=[CH:24][C:7]2[N:8](...,CC[O:16][C:14]([CH:13]=[C:12]1[C:6]2=[C:7]([N:...,[OH-],[K+],Cl,NaN,NaN,NaN,NaN,NaN,NaN,CCO,NaN,NaN,[CH2:1]([CH3:2])[S:3][C:4]1=[CH:5][C:6]2=[C:7]...,NaN,78.1
2,[H-].[Al+3].[Li+].[H-].[H-].[H-].[CH2:7]([S:9]...,[H-],[Al+3],[Li+],[H-],[H-],[H-],O=[C:20]([CH2:19][C:18]1=[CH:17][C:16]2=[C:15]...,CCOC(C)=O,[OH-],[Na+],C1CCOC1,NaN,NaN,[CH2:7]([CH3:8])[S:9][C:10]1=[CH:11][C:12]2=[C...,NaN,82.1
3,[NH2:1][C:2]1[CH:3]=[CH:4][C:5]2[N:9]=[C:8]3[C...,[NH2:1][C:2]1=[CH:13][C:6]2=[C:5]([CH:4]=[CH:3...,CC(=O)O[C:14]([CH3:15])=[O:16],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C1=CC=CC=C1,NaN,NaN,[NH:1]([C:2]1=[CH:13][C:6]2=[C:5]([CH:4]=[CH:3...,NaN,67.9
4,[C:1]([NH:5][NH:6][C:7]([C:12]#[N:13])([CH:9]1...,[C:1]([CH3:2])([CH3:3])([CH3:4])[NH:5][NH:6][C...,ClCCl,BrBr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,NaN,[C:1]([CH3:2])([CH3:3])([CH3:4])[N:5]=[N:6][C:...,NaN,81.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779098,[Br:1][C:2]1[S:3][C:4]([Br:16])=[CH:5][C:6]=1[...,Br[CH2:14][CH2:13][CH2:12][CH2:11][CH2:10][CH2...,[OH:17][C:18]1=[CH:19][CH:20]=[C:21](/[CH:24]=...,O=C([O-])[O-],[K+],[K+],NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[Br:1][C:2]1=[C:6]([CH2:7][CH2:8][CH2:9][CH2:1...,NaN,88.0
779099,[Br:1][C:2]1[S:3][C:4]([Br:16])=[CH:5][C:6]=1[...,BrCC[CH2:12][CH2:11][CH2:10][CH2:9][CH2:8][CH2...,[F:17][C:18]1=[CH:19][CH:20]=[C:21](/[CH:24]=[...,O=C([O-])[O-],[K+],[K+],NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[Br:1][C:2]1=[C:6]([CH2:7][CH2:8][CH2:9][CH2:1...,NaN,85.0
779100,[OH:1][C:2]1[CH:7]=[CH:6][C:5](/[CH:8]=[CH:9]/...,[OH:1][C:2]1=[CH:3][CH:4]=[C:5](/[CH:8]=[CH:9]...,Br[CH2:21][CH2:20][CH2:19][CH2:18][CH2:17][CH2...,O=C([O-])[O-],[K+],[K+],NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[O:1]([C:2]1=[CH:7][CH:6]=[C:5](/[CH:8]=[CH:9]...,NaN,92.3
779101,[Br:1][C:2]1[CH:14]=[CH:13][C:12]2[C:11]3[C:6]...,[Br:1][C:2]1=[CH:3][C:4]2=[C:12]([C:11]3=[CH:1...,[H-],[Na+],Br[CH2:19][CH2:20][CH2:21][CH2:22][CH2:23][CH2...,NaN,NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[Br:1][C:2]1=[CH:3][C:4]2=[C:12]([C:11]3=[CH:1...,NaN,50.4


,reactions,interm_0,interm_1,interm_2,interm_3,interm_4,interm_5,interm_6,interm_7,interm_8,interm_9,agents_0,agents_1,agents_2,product_0,product_1,Yield
0,[CH2:1]([S:3][C:4]1[CH:26]=[CH:25][C:7]2[N:8](...,O[C:12]1([CH2:13][C:14](=[O:15])[O:16][CH2:17]...,Cl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCO,NaN,NaN,[CH2:1]([CH3:2])[S:3][C:4]1=[CH:5][C:6]2=[C:7]...,NaN,82.0
1,[CH2:1]([S:3][C:4]1[CH:25]=[CH:24][C:7]2[N:8](...,CC[O:16][C:14]([CH:13]=[C:12]1[C:6]2=[C:7]([N:...,[OH-],[K+],Cl,NaN,NaN,NaN,NaN,NaN,NaN,CCO,NaN,NaN,[CH2:1]([CH3:2])[S:3][C:4]1=[CH:5][C:6]2=[C:7]...,NaN,78.1
2,[H-].[Al+3].[Li+].[H-].[H-].[H-].[CH2:7]([S:9]...,[H-],[Al+3],[Li+],[H-],[H-],[H-],O=[C:20]([CH2:19][C:18]1=[CH:17][C:16]2=[C:15]...,CCOC(C)=O,[OH-],[Na+],C1CCOC1,NaN,NaN,[CH2:7]([CH3:8])[S:9][C:10]1=[CH:11][C:12]2=[C...,NaN,82.1
3,[NH2:1][C:2]1[CH:3]=[CH:4][C:5]2[N:9]=[C:8]3[C...,[NH2:1][C:2]1=[CH:13][C:6]2=[C:5]([CH:4]=[CH:3...,CC(=O)O[C:14]([CH3:15])=[O:16],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C1=CC=CC=C1,NaN,NaN,[NH:1]([C:2]1=[CH:13][C:6]2=[C:5]([CH:4]=[CH:3...,NaN,67.9
4,[C:1]([NH:5][NH:6][C:7]([C:12]#[N:13])([CH:9]1...,[C:1]([CH3:2])([CH3:3])([CH3:4])[NH:5][NH:6][C...,ClCCl,BrBr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,NaN,[C:1]([CH3:2])([CH3:3])([CH3:4])[N:5]=[N:6][C:...,NaN,81.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779098,[Br:1][C:2]1[S:3][C:4]([Br:16])=[CH:5][C:6]=1[...,Br[CH2:14][CH2:13][CH2:12][CH2:11][CH2:10][CH2...,[OH:17][C:18]1=[CH:19][CH:20]=[C:21](/[CH:24]=...,O=C([O-])[O-],[K+],[K+],NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[Br:1][C:2]1=[C:6]([CH2:7][CH2:8][CH2:9][CH2:1...,NaN,88.0
779099,[Br:1][C:2]1[S:3][C:4]([Br:16])=[CH:5][C:6]=1[...,BrCC[CH2:12][CH2:11][CH2:10][CH2:9][CH2:8][CH2...,[F:17][C:18]1=[CH:19][CH:20]=[C:21](/[CH:24]=[...,O=C([O-])[O-],[K+],[K+],NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[Br:1][C:2]1=[C:6]([CH2:7][CH2:8][CH2:9][CH2:1...,NaN,85.0
779100,[OH:1][C:2]1[CH:7]=[CH:6][C:5](/[CH:8]=[CH:9]/...,[OH:1][C:2]1=[CH:3][CH:4]=[C:5](/[CH:8]=[CH:9]...,Br[CH2:21][CH2:20][CH2:19][CH2:18][CH2:17][CH2...,O=C([O-])[O-],[K+],[K+],NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[O:1]([C:2]1=[CH:7][CH:6]=[C:5](/[CH:8]=[CH:9]...,NaN,92.3
779101,[Br:1][C:2]1[CH:14]=[CH:13][C:12]2[C:11]3[C:6]...,[Br:1][C:2]1=[CH:3][C:4]2=[C:12]([C:11]3=[CH:1...,[H-],[Na+],Br[CH2:19][CH2:20][CH2:21][CH2:22][CH2:23][CH2...,NaN,NaN,NaN,NaN,NaN,NaN,CN(C)C=O,Cl,NaN,[Br:1][C:2]1=[CH:3][C:4]2=[C:12]([C:11]3=[CH:1...,NaN,50.4


In [12]:
for i in tqdm(data.columns[10:11]):
    for j in range(0,780001, 10000):
        rd = RDKit_2D(data[i].loc[j:j+10000-1])
        rd.compute_2Drdkit(f"{i}_{j}={j+10000}")

        rd = ECFP6(data[i].loc[j:j+10000-1])
        rd.compute_ECFP6(f"{i}_{j}={j+10000}")

        rd = MACCS(data[i].loc[j:j+10000-1])
        rd.compute_MACCS(f"{i}_{j}={j+10000}")

    

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

[23:27:13] WARNING: not removing hydrogen atom without neighbors
[23:27:17] WARNING: not removing hydrogen atom without neighbors
[23:27:23] WARNING: not removing hydrogen atom without neighbors


  0%|          | 0/10000 [00:00<?, ?it/s]

[23:27:26] WARNING: not removing hydrogen atom without neighbors
[23:27:26] WARNING: not removing hydrogen atom without neighbors
[23:27:26] WARNING: not removing hydrogen atom without neighbors
[23:27:26] WARNING: not removing hydrogen atom without neighbors
[23:27:27] WARNING: not removing hydrogen atom without neighbors
[23:27:27] WARNING: not removing hydrogen atom without neighbors


...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]